In [ ]:
pip install torch

In [ ]:
pip install ultralytics

In [ ]:
pip install opencv-python

In [ ]:
pip install pandas

In [ ]:
pip install pathlib

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

In [12]:
import torch
import torchvision

In [2]:
# Шаг 0. Проверка работы CUDA
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 4070 SUPER


# Расположение файлов в директории
<pre>
│─ data.yaml
│─ model.pt <i>(Обученная модель YOLOv8s)</i>
│─ solution/
│   ├─ metadata.json
│   ├─ model.pt
│   ├─ solution.py <i>(Скрипт, реализующий решение)</i>
│   └─ solution.zip <i>(Готовый архив решения, который загружается в систему)</i>
│─ test/ <i>(Директория с изображениями для ручного тестирования модели)</i>
│─ dataset_additional_training/ <i>(Датасет <b>дообучения</b> модели)</i>
│   ├─ images/
│   │   ├─ train/
│   │   └─ val/
│   └─ labels/
│       ├─ train/
│       └─ val/
│─ additional_training_data/ <i>(Пары для сбора датасета <b>дообучения</b> модели)</i>
│   ├─ images/
│   └─ labels/
│─ training_data/ <i>(Пары для сбора датасета <b>обучения</b> модели)</i>
│   ├─ images/
│   └─ labels/
│─ runs/ <i>(Результаты <b>обучения</b> модели)</i>
│   └─ detect/
└─ dataset/ <i>(Основной датасет <b>обучения</b> модели)</i>
    ├─ images/
    │   ├─ train/
    │   └─ val/
    └─ labels/
        ├─ train/
        └─ val/
</pre>

# Обучение модели YOLOv8s

In [2]:
# Шаг 1. Подготовка датасета. 
# Шаг 1.1 Переименование расширения с .JPG на .jpg
import os

IMAGES_DIR = 'C:/Human_rescue/additional_training_data/images'

def rename_jpg_to_lowercase():
    print(f"Обрабатываю директорию: {IMAGES_DIR}")
    
    # Проверяем, существует ли директория
    if not os.path.exists(IMAGES_DIR):
        print("Ошибка: Директория не существует!")
        return
    
    count = 0
    
    for filename in os.listdir(IMAGES_DIR):
        if filename.endswith('.JPG'):
            old_path = os.path.join(IMAGES_DIR, filename)
            new_filename = filename[:-4] + '.jpg'
            new_path = os.path.join(IMAGES_DIR, new_filename)
            
            try:
                os.rename(old_path, new_path)
                print(f"Переименовано: {filename} -> {new_filename}")
                count += 1
            except Exception as e:
                print(f"Ошибка при переименовании {filename}: {e}")
    
    print(f"\nГотово! Переименовано файлов: {count}")

if __name__ == "__main__":
    rename_jpg_to_lowercase()

In [4]:
# Шаг 1. Подготовка датасета. 
# Шаг 1.2 Проверка наличия пары image-label

def check_matching_files(images_dir, labels_dir, image_ext='.jpg', label_ext='.txt'):
    if not os.path.exists(images_dir):
        raise FileNotFoundError(f"Папка с изображениями не найдена: {os.path.abspath(images_dir)}")
    if not os.path.exists(labels_dir):
        raise FileNotFoundError(f"Папка с разметкой не найдена: {os.path.abspath(labels_dir)}")

    image_files = [f for f in os.listdir(images_dir) if f.endswith(image_ext)]
    label_files = [f for f in os.listdir(labels_dir) if f.endswith(label_ext)]

    if not image_files:
        print(f"Ошибка! В папке {images_dir} нет файлов с расширением {image_ext}")
    if not label_files:
        print(f"Ошибка! В папке {labels_dir} нет файлов с расширением {label_ext}")

    image_names = {os.path.splitext(f)[0] for f in image_files}
    label_names = {os.path.splitext(f)[0] for f in label_files}

    images_without_labels = image_names - label_names
    labels_without_images = label_names - image_names

    report = {
        'total_images': len(image_files),
        'total_labels': len(label_files),
        'images_without_labels': len(images_without_labels),
        'labels_without_images': len(labels_without_images),
        'missing_labels': sorted(f"{name}{image_ext}" for name in images_without_labels),
        'orphan_labels': sorted(f"{name}{label_ext}" for name in labels_without_images),
        'all_matched': not images_without_labels and not labels_without_images
    }
    return report

if __name__ == "__main__":
    # Проверка наличия пар до деления на train и val
    # IMAGES_DIR = 'C:/Human_rescue/additional_training_data/images'
    # LABELS_DIR = 'C:/Human_rescue/additional_training_data/labels'

    # Проверка наличия пар после деления на train и val
    # IMAGES_DIR = 'C:/Human_rescue/dataset/images/train'
    # LABELS_DIR = 'C:/Human_rescue/dataset/labels/train'

    # Датасет дообучения
    IMAGES_DIR = 'C:/Human_rescue/dataset_additional_training/images/train'
    LABELS_DIR = 'C:/Human_rescue/dataset_additional_training/labels/train'
    
    try:
        report = check_matching_files(IMAGES_DIR, LABELS_DIR)
        
        print(f"Папка с изображениями: {os.path.abspath(IMAGES_DIR)}")
        print(f"Папка с разметкой: {os.path.abspath(LABELS_DIR)}\n")
        
        print(f"Всего изображений: {report['total_images']}")
        print(f"Всего файлов разметки: {report['total_labels']}\n")

        if report['all_matched']:
            print("Успешно! Все файлы соответствуют!")
        else:
            if report['images_without_labels']:
                print(f"Найдено {report['images_without_labels']} изображений без разметки:")
                for img in report['missing_labels']:
                    print(f"  - {img}")
            
            if report['labels_without_images']:
                print(f"\nНайдено {report['labels_without_images']} файлов разметки без изображений:")
                for lbl in report['orphan_labels']:
                    print(f"  - {lbl}")
    
    except Exception as e:
        print(f"Ошибка: {e}")

Папка с изображениями: C:\Human_rescue\dataset_additional_training\images\train
Папка с разметкой: C:\Human_rescue\dataset_additional_training\labels\train

Всего изображений: 800
Всего файлов разметки: 800

Успешно! Все файлы соответствуют!


In [3]:
# Шаг 1. Подготовка датасета. 
# Шаг 1.3 Деление датасета на train и val
import shutil
import random
import time

def split_dataset(
    images_dir,
    labels_dir,
    # "output_root": "dataset",
    output_root="dataset_additional_training",
    train_ratio=0.8,
    image_ext=".jpg",
    label_ext=".txt",
    seed=None,
    shuffle_rounds=3,
):
    if seed is not None:
        random.seed(seed)
    else:
        random.seed(int(time.time() * 1_000_000) % (2**32))

    # Создаем папки (если нужно)
    os.makedirs(os.path.join(output_root, "images", "train"), exist_ok=True)
    os.makedirs(os.path.join(output_root, "images", "val"), exist_ok=True)
    os.makedirs(os.path.join(output_root, "labels", "train"), exist_ok=True)
    os.makedirs(os.path.join(output_root, "labels", "val"), exist_ok=True)

    # Получаем список парных файлов
    image_files = [f for f in os.listdir(images_dir) if f.endswith(image_ext)]
    label_files = [f for f in os.listdir(labels_dir) if f.endswith(label_ext)]

    image_names = {os.path.splitext(f)[0] for f in image_files}
    label_names = {os.path.splitext(f)[0] for f in label_files}
    paired_names = list(image_names & label_names)

    # Усиленное перемешивание
    for _ in range(shuffle_rounds):
        random.shuffle(paired_names)
        time.sleep(0.01)

    split_idx = int(len(paired_names) * train_ratio)
    train_names = paired_names[:split_idx]
    val_names = paired_names[split_idx:]

    print(f"Всего парных файлов: {len(paired_names)}")
    print(f"Train: {len(train_names)} ({(len(train_names)/len(paired_names)*100):.1f}%)")
    print(f"Val: {len(val_names)} ({(len(val_names)/len(paired_names)*100):.1f}%)")

    # Копирование файлов
    try:
        from tqdm import tqdm
        train_names = tqdm(train_names, desc="Копирование train")
        val_names = tqdm(val_names, desc="Копирование val")
    except ImportError:
        pass

    for name in train_names:
        shutil.copy(
            os.path.join(images_dir, f"{name}{image_ext}"),
            os.path.join(output_root, "images", "train", f"{name}{image_ext}"),
        )
        shutil.copy(
            os.path.join(labels_dir, f"{name}{label_ext}"),
            os.path.join(output_root, "labels", "train", f"{name}{label_ext}"),
        )

    for name in val_names:
        shutil.copy(
            os.path.join(images_dir, f"{name}{image_ext}"),
            os.path.join(output_root, "images", "val", f"{name}{image_ext}"),
        )
        shutil.copy(
            os.path.join(labels_dir, f"{name}{label_ext}"),
            os.path.join(output_root, "labels", "val", f"{name}{label_ext}"),
        )

    print("\nГотово! Результат:")
    print(f"│── images/")
    print(f"│   ├── train/ ({len(train_names)} файлов)")
    print(f"│   └── val/ ({len(val_names)} файлов)")
    print(f"└── labels/")
    print(f"    ├── train/ ({len(train_names)} файлов)")
    print(f"    └── val/ ({len(val_names)} файлов)")

if __name__ == "__main__":
    config = {
        "images_dir": "C:/Human_rescue/additional_training_data/images",
        "labels_dir": "C:/Human_rescue/additional_training_data/labels",
        # "output_root": "C:/Human_rescue/dataset",
        "output_root": "C:/Human_rescue/dataset_additional_training",
        "train_ratio": 0.8,
        "seed": None,
        "shuffle_rounds": 5,
    }

    split_dataset(**config)

Всего парных файлов: 1000
Train: 800 (80.0%)
Val: 200 (20.0%)


Копирование train: 100%|████████████████████████████████████████████████████████████| 800/800 [00:07<00:00, 104.48it/s]

Копирование val: 100%|███████████████████████████████████████████████████████████████| 200/200 [00:09<00:00, 20.82it/s]


Готово! Результат:
│── images/
│   ├── train/ (800 файлов)
│   └── val/ (200 файлов)
└── labels/
    ├── train/ (800 файлов)
    └── val/ (200 файлов)


In [1]:
from ultralytics import YOLO

#model = YOLO('yolov8s.yaml') # Изначальная модель YOLO
model = YOLO('best.pt'); # Дообучение модели

model.train(
    data='data.yaml',
    epochs=100,
    imgsz=1280,
    batch=10,
    device='0',
    lr0=0.001,
    amp=True,
    workers=6,
    patience=20,
    cos_lr=True,
    optimizer='AdamW',
    weight_decay=0.001,
    box=9.0,
    cls=0.5,
    dfl=2.5,
    single_cls=True,
    warmup_epochs=5,
    warmup_momentum=0.8,
    hsv_h=0.1, hsv_s=0.1, hsv_v=0.1,
    degrees=10.0, flipud=0.5, fliplr=0.5,
    mosaic=0.2,
)

New https://pypi.org/project/ultralytics/8.3.175 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.172  Python-3.11.5 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=10, bgr=0.0, box=9.0, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=10.0, deterministic=True, device=0, dfl=2.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.1, hsv_s=0.1, hsv_v=0.1, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=best.pt, momentum=0.937, mosaic=0.2, multi_scale=False, name=train38, nbs=64, nms=False, opset=None, optimize=False, o

train: Scanning C:\Human_rescue\dataset\labels\train... 800 images, 232 backgrounds, 0 corrupt: 100%|██████████| 800/80

train: New cache created: C:\Human_rescue\dataset\labels\train.cache


val: Fast image access  (ping: 0.10.0 ms, read: 786.8248.9 MB/s, size: 7703.2 KB)


val: Scanning C:\Human_rescue\dataset\labels\val... 200 images, 62 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00

...

                   all        200        353      0.937      0.882      0.929      0.568
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 78, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



98 epochs completed in 0.681 hours.
Optimizer stripped from runs\detect\train38\weights\last.pt, 22.6MB
Optimizer stripped from runs\detect\train38\weights\best.pt, 22.6MB

Validating runs\detect\train38\weights\best.pt...
Ultralytics 8.3.172  Python-3.11.5 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
YOLOv8s summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02


                   all        200        353      0.932      0.887      0.933       0.59
Speed: 3.7ms preprocess, 4.2ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\train38


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000023D10806F10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [4]:
model.save()

In [58]:
# Проверка работы модели на ручной тестовой выборке
model = YOLO('model.pt')  

img = cv2.imread('C:/Human_rescue/test/test_image.jpg')  

results = model.predict(img, classes=[0], conf=0.5)

results[0].show()
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 864x1280 1 person, 33.8ms
Speed: 6.5ms preprocess, 33.8ms inference, 1.4ms postprocess per image at shape (1, 3, 864, 1280)
